<a href="https://colab.research.google.com/github/Alphambarushimana/Sustainability/blob/main/New_M2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and libraries

In [43]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

In [44]:
# explainability (why did the model say it's related to this author)
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
!pip install pybliometrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import pandas as pd
import numpy as np
import preprocessor as prepro # text prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

from pybliometrics.scopus import AbstractRetrieval

In [47]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

# Data loading and preparation

In [48]:
data = pd.read_csv("https://raw.githubusercontent.com/Alphambarushimana/Sustainability/main/scopuswithlabels.csv")

In [49]:
data

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Open Access,Source,EID
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,2022,Bulletin of Mathematical Biology,84,12,136,NaN,NaN,...,Department of Mathematics and Applied Mathemat...,"Chapwanya, M., Department of Mathematics and A...",The tenth Ebola outbreak in the Democratic Rep...,Basic reproduction number; Conflict dynamics; ...,basic reproduction number; biological model; D...,Article,Final,NaN,Scopus,2-s2.0-85140065004
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,2022,International Journal of Mental Health Systems,16,1,49,NaN,NaN,...,"Department of Psychiatry, McGill University an...","Piat, M., Department of Psychiatry, McGill Uni...",Background: Implementing mental health recover...,Conceptual framework; Guidelines; Impact; Impl...,adult; article; conceptual framework; content ...,Article,Final,NaN,Scopus,2-s2.0-85139748940
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,2022,BMC Medical Informatics and Decision Making,22,1,266,NaN,NaN,...,"Telemedicine Research Center, National Researc...","Velayati, F., Telemedicine Research Center, Na...",Background: Telehealth services can utilize va...,Business; Health service marketing; Telehealth...,adult; article; comparative effectiveness; Del...,Article,Final,NaN,Scopus,2-s2.0-85139435002
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,2022,Mitigation and Adaptation Strategies for Globa...,27,8,52,NaN,NaN,...,"LISAH, University Montpellier, INRAE, IRD, Ins...","Pastor, A.V., LISAH, University Montpellier, I...",Scenarios serve science by testing the sensiti...,Experts; Land degradation; Land use and manage...,agricultural soil; environmental protection; e...,Article,Final,NaN,Scopus,2-s2.0-85138219727
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,2022,BMC Neurology,22,1,346,NaN,NaN,...,"Department of Surgery, Korle Bu Teaching Hospi...","Abdullah, H.M., Department of Surgery, Korle B...",Background: Chronic subdural haematoma (CSDH) ...,Association; Chronic; Haematoma; Predictors; R...,anticoagulant agent; antithrombocytic agent; w...,Article,Final,NaN,Scopus,2-s2.0-85137878264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,"Armeanu D.S., Vintilă G., Gherghina S.C.",26434149400;26434536500;56046530600;,Empirical study towards the drivers of sustain...,2018,Sustainability (Switzerland),10,1,4,NaN,NaN,...,"Department of Finance, The Bucharest Universit...","Armeanu, D.S., Department of Finance, The Buch...",This study aims at empirically investigating t...,Economic growth; EU-28; Fixed and random effec...,economic growth; employment; European Union; h...,Article,Final,NaN,Scopus,2-s2.0-85038843701
1630,"Abaunza F., Hameri A.-P., Niemi T.",56060899300;7003735163;57215042589;,EEUI: a new measure to monitor and manage ener...,2018,International Journal of Productivity and Perf...,67,1,NaN,111,127,...,"Faculty of Business and Economics, University ...","Abaunza, F., Faculty of Business and Economics...",Purpose: Data centers (DCs) are similar to tra...,Data centres; Energy efficiency; Energy propor...,NaN,Article,Final,NaN,Scopus,2-s2.0-85038831501
1631,"Zhang Y., Lin Z.",57840767400;54894812400;,Predicting the helpfulness of online product r...,2018,Electronic Commerce Research and Applications,27,NaN,NaN,1,10,...,"School of

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Authors                    1634 non-null   object 
 1   Author(s) ID               1634 non-null   object 
 2   Title                      1634 non-null   object 
 3   Year                       1634 non-null   int64  
 4   Source title               1634 non-null   object 
 5   Volume                     1219 non-null   object 
 6   Issue                      930 non-null    object 
 7   Art. No.                   480 non-null    object 
 8   Page start                 1046 non-null   object 
 9   Page end                   1026 non-null   object 
 10  Page count                 47 non-null     float64
 11  Cited by                   1005 non-null   float64
 12  DOI                        1428 non-null   object 
 13  Link                       1634 non-null   objec

In [51]:
data = data[['Authors', 'Author(s) ID','Title', 'Abstract','Year', 'Source title']]

In [52]:
data.isnull().sum()

Authors         0
Author(s) ID    0
Title           0
Abstract        0
Year            0
Source title    0
dtype: int64

In [53]:
data.dropna(subset=['Author(s) ID'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [54]:
data.head()

,Authors,Author(s) ID,Title,Abstract,Year,Source title
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,The tenth Ebola outbreak in the Democratic Rep...,2022,Bulletin of Mathematical Biology
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,Background: Implementing mental health recover...,2022,International Journal of Mental Health Systems
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,Background: Telehealth services can utilize va...,2022,BMC Medical Informatics and Decision Making
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,Scenarios serve science by testing the sensiti...,2022,Mitigation and Adaptation Strategies for Globa...
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,Background: Chronic subdural haematoma (CSDH) ...,2022,BMC Neurology


# Topic modelling

In [55]:
# Creating a new column.
data ['text'] = data['Title'] + '. ' + data['Abstract']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
data['text_clean'] = data['text'].map(lambda t: prepro.clean(t))
data['text_clean'] = data['text_clean'].str.replace('#','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline
clean_text = []

pbar = tqdm.tqdm(total=len(data['text_clean']),position=0, leave=True)

for text in nlp.pipe(data['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

100%|██████████| 1634/1634 [1:12:41<00:00,  2.67s/it]
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
 99%|█████████▉| 1618/1634 [00:35<00:00, 100.87it/s]

In [58]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [59]:
# apply all prepro-pipeline to texts
data['text_clean'] = text_prepro(data['text'])

100%|██████████| 1634/1634 [00:24<00:00, 66.84it/s] 
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
data

,Authors,Author(s) ID,Title,Abstract,Year,Source title,text,text_clean
0,"Chapwanya M., Lubuma J., Terefe Y., Tsanou B.",26026788700;6602950766;54783401400;25823549000;,Analysis of War and Conflict Effect on the Tra...,The tenth Ebola outbreak in the Democratic Rep...,2022,Bulletin of Mathematical Biology,Analysis of War and Conflict Effect on the Tra...,analysis war conflict effect transmission dyna...
1,"Piat M., Wainwright M., Rivest M.-P., Sofouli ...",6604070039;35090631400;56871264100;18635052000...,The impacts of implementing recovery innovatio...,Background: Implementing mental health recover...,2022,International Journal of Mental Health Systems,The impacts of implementing recovery innovatio...,impacts implementing recovery innovations conc...
2,"Velayati F., Ayatollahi H., Hemmat M., Dehghan R.",57219486899;55991128900;57170342800;57365458900;,The 4P telehealth business framework for Iran,Background: Telehealth services can utilize va...,2022,BMC Medical Informatics and Decision Making,The 4P telehealth business framework for Iran....,p telehealth business framework iran backgroun...
3,"Pastor A.V., Nunes J.P., Ciampalini R., Bahri ...",55927118200;7102878032;6506995140;7004276837;5...,ScenaLand: a simple methodology for developing...,Scenarios serve science by testing the sensiti...,2022,Mitigation and Adaptation Strategies for Globa...,ScenaLand: a simple methodology for developing...,scenaland simple methodology developing land u...
4,"Abdullah H.M., Dakurah T., Akoto H., Abaidoo B...",57887451300;12140148900;57213456448;5623694480...,Predictors of recurrence of chronic subdural h...,Background: Chronic subdural haematoma (CSDH) ...,2022,BMC Neurology,Predictors of recurrence of chronic subdural h...,predictors recurrence chronic subdural haemato...
...,...,...,...,...,...,...,...,...
1629,"Armeanu D.S., Vintilă G., Gherghina S.C.",26434149400;26434536500;56046530600;,Empirical study towards the drivers of sustain...,This study aims at empirically investigating t...,2018,Sustainability (Switzerland),Empirical study towards the drivers of sustain...,empirical study drivers sustainable economic g...
1630,"Abaunza F., Hameri A.-P., Niemi T.",56060899300;7003735163;57215042589;,EEUI: a new measure to monitor and manage ener...,Purpose: Data centers (DCs) are similar to tra...,2018,International Journal of Productivity and Perf...,EEUI: a new measure to monitor and manage ener...,eeui new measure monitor manage energy efficie...
1631,"Zhang Y., Lin Z.",57840767400;54894812400;,Predicting the helpfulness of online product r...,Identifying helpful reviews from massive revie...,2018,Electronic Commerce Research and Applications,Predicting the helpfulness of online product r...,predicting helpfulness online product reviews ...
1632,"Greening H., Swann R., St. Pé K., Testroet-Ber...",6602993313;36025859800;15038078600;57196705444...,Local implementation of a national program: Th...,The US Environmental Protection Agency's Natio...,2018,Marine Policy,Local implementation of a national program: Th...,local implementation national program national...


In [61]:
# preprocess texts (we need tokens)
tokens = []

for summary in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in summary 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [62]:
data['tokens'] = tokens

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=3, no_above=0.2, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data['tokens']]

In [64]:
dictionary.id2token

{}

In [65]:
corpus[1]

[(22, 1),
 (23, 3),
 (26, 9),
 (28, 1),
 (49, 2),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 2),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 2),
 (67, 5),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 3),
 (72, 4),
 (73, 7),
 (74, 2),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 2),
 (80, 3),
 (81, 1),
 (82, 1),
 (83, 3),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 2),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 2),
 (96, 2),
 (97, 2),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 8),
 (102, 1),
 (103, 1),
 (104, 2),
 (105, 3),
 (106, 1),
 (107, 1),
 (108, 3),
 (109, 1),
 (110, 1),
 (111, 3),
 (112, 1),
 (113, 1)]

In [66]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=5, workers = 4, passes=10)

In [67]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [68]:
 # Let's Visualize
pyLDAvis.display(lda_display)

# EDA and dynamic effects (over time)

# Sparse paper-keywords matrix

In [69]:
data_1 = pd.read_csv("https://raw.githubusercontent.com/Alphambarushimana/Sustainability/main/scopuswithlabels.csv")

In [70]:
data_1 = data_1[['Authors', 'Author(s) ID','Title', 'Abstract','Year', 'Source title', 'Author Keywords']]

In [71]:
data_1.isnull().sum()

Authors              0
Author(s) ID         0
Title                0
Abstract             0
Year                 0
Source title         0
Author Keywords    284
dtype: int64

In [72]:
data_1.dropna(subset=['Author(s) ID'], inplace=True)

In [73]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [74]:
# Creating a new column.
data_1['text'] = data_1['Author Keywords']

In [75]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1634 entries, 0 to 1633
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Authors          1634 non-null   object
 1   Author(s) ID     1634 non-null   object
 2   Title            1634 non-null   object
 3   Abstract         1634 non-null   object
 4   Year             1634 non-null   int64 
 5   Source title     1634 non-null   object
 6   Author Keywords  1350 non-null   object
 7   text             1350 non-null   object
dtypes: int64(1), object(7)
memory usage: 114.9+ KB


In [76]:
data_1['text_clean'] = data_1['text'].map(lambda t: prepro.clean(t))
data_1['text_clean'] = data_1['text_clean'].str.replace('#','')

TypeError: ignored

In [ ]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline

clean_text = []

pbar = tqdm.tqdm(total=len(data_1['text_clean']),position=0, leave=True)

for text in nlp.pipe(data_1['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop # stop words such as: and, like... to reduce diversity and normlize vocabulary.
         and not token.is_punct] # 

  clean_text.append(" ".join(txt))

  pbar.update(1)

In [ ]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [ ]:
# apply all prepro-pipeline to texts
data_1['text_clean'] = text_prepro(data_1['text'])

In [ ]:
data_1

-----------------------------------------------------------------------

In [ ]:
# load models and utility
from gensim.models import LsiModel, TfidfModel
from gensim.matutils import corpus2dense

In [ ]:
# fit tfidf weights
tfidf = TfidfModel(corpus)

In [ ]:
# apply tfidf to data
corpus_tfidf = tfidf[corpus]

In [ ]:
corpus[2]

In [ ]:
corpus_tfidf[2]

In [ ]:
data['tokens'].iloc[2]

In [ ]:
# train LSI model
lsi = LsiModel(corpus_tfidf, num_topics=10, id2word=dictionary)

In [ ]:
lsi.print_topics()

In [ ]:
# apply lsi (that's equivalent to SVD fit/transform in sklearn)
corpus_lsi = lsi[corpus_tfidf]

In [ ]:
corpus_lsi[0]

In [ ]:
# turning it back into a matrix
lsi_matrix = corpus2dense(corpus_lsi, num_terms = 10)

In [ ]:
# getting the top topic-index per doc.
np.argmax(lsi_matrix.T[0])

In [ ]:
ab = AbstractRetrieval("2-s2.0-85068268027", view='FULL')